# Preparação de Dados Médicos para Fine-Tuning

Este notebook processa o dataset `ori_pqal.json` para criar dados formatados para fine-tuning de modelos LLM (LLaMA/Falcon) em domínio médico.

## 📋 Objetivos do Pipeline:

1. **Carregamento**: Ler o dataset médico original (ori_pqal.json) com estrutura complexa
2. **Anonimização**: Remover dados sensíveis (datas, IDs, telefones, emails) para conformidade com LGPD/HIPAA
3. **Transformação**: Converter estrutura JSON aninhada em formato de instrução para fine-tuning
4. **Validação**: Verificar integridade e qualidade dos dados processados
5. **Exportação**: Salvar dataset processado em formato JSON pronto para treinamento

## 🔄 Ordem de Execução:

Execute as células **sequencialmente** (de cima para baixo) para garantir que todas as dependências estejam carregadas corretamente.


In [ ]:
# ============================================================================
# ETAPA 0: IMPORTAÇÃO DE BIBLIOTECAS
# ============================================================================
# Esta célula importa todas as bibliotecas necessárias para o processamento
# Execute esta célula PRIMEIRO antes de qualquer outra operação

import json      # Para leitura e escrita de arquivos JSON
import re        # Para expressões regulares (anonimização de dados)
from pathlib import Path  # Para manipulação de caminhos de arquivos

print("✅ Bibliotecas importadas com sucesso!")
print("   - json: Para manipulação de arquivos JSON")
print("   - re: Para anonimização com expressões regulares")
print("   - Path: Para gerenciamento de caminhos de arquivos")


In [ ]:
# ============================================================================
# ETAPA 1: CARREGAMENTO DO DATASET ORIGINAL
# ============================================================================
# Esta etapa lê o arquivo ori_pqal.json que contém dados médicos estruturados
# do PubMedQA (dataset de perguntas e respostas médicas baseadas em evidências)
#
# Estrutura do dataset:
#   - Chave: ID único do artigo PubMed (ex: "21645374")
#   - Valor: Objeto com QUESTION, CONTEXTS, LONG_ANSWER, MESHES, etc.
#
# IMPORTANTE: Execute esta célula ANTES de processar os dados

def load_medical_dataset(file_path):
    """
    Carrega o dataset médico do arquivo JSON
    
    Args:
        file_path: Caminho relativo ou absoluto para o arquivo ori_pqal.json
        
    Returns:
        Dicionário Python com estrutura: {id_artigo: {QUESTION, CONTEXTS, ...}}
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# Define o caminho para o dataset original
# Ajuste este caminho se o arquivo estiver em outro local
input_file = '../context/pubmedqa-master/data/ori_pqal.json'

# Carrega todos os dados do arquivo JSON na memória
# Isso pode levar alguns segundos dependendo do tamanho do arquivo
raw_data = load_medical_dataset(input_file)

# Exibe informações sobre o dataset carregado
print("=" * 80)
print("📦 DATASET CARREGADO COM SUCESSO")
print("=" * 80)
print(f"Total de entradas médicas: {len(raw_data)}")
print(f"Exemplo de ID (chave): {list(raw_data.keys())[0]}")
print(f"\nEstrutura de uma entrada:")
sample_key = list(raw_data.keys())[0]
sample_entry = raw_data[sample_key]
print(f"  - QUESTION: {sample_entry.get('QUESTION', 'N/A')[:80]}...")
print(f"  - CONTEXTS: {len(sample_entry.get('CONTEXTS', []))} contextos")
print(f"  - LONG_ANSWER: {len(sample_entry.get('LONG_ANSWER', ''))} caracteres")
print(f"  - MESHES: {len(sample_entry.get('MESHES', []))} termos")
print("=" * 80)


In [ ]:
# ============================================================================
# ETAPA 2: FUNÇÃO DE ANONIMIZAÇÃO DE DADOS SENSÍVEIS
# ============================================================================
# Esta etapa define a função que remove informações que possam identificar
# pacientes ou violar privacidade (conformidade com LGPD/HIPAA)
#
# Por que anonimizar?
#   - Proteção de dados pessoais (LGPD no Brasil, HIPAA nos EUA)
#   - Prevenção de vazamento de informações sensíveis
#   - Necessário para ambientes hospitalares e pesquisa médica
#
# IMPORTANTE: Esta função será usada em TODAS as etapas de processamento
#             que envolvem texto com dados de pacientes

def anonymize_text(text):
    """
    Anonimiza texto removendo padrões que possam identificar pacientes
    
    Esta função usa expressões regulares (regex) para encontrar e substituir:
    - Datas específicas → [DATA]
    - IDs de pacientes → [PACIENTE_ID]
    - Telefones → [TELEFONE]
    - Emails → [EMAIL]
    
    Args:
        text: String de texto que pode conter dados sensíveis
        
    Returns:
        String com dados sensíveis substituídos por placeholders genéricos
    """
    # Verifica se o texto é uma string válida
    if not isinstance(text, str):
        return text
    
    # PATTERN 1: Remove datas no formato DD/MM/YYYY ou MM/DD/YYYY
    # Exemplo: "15/03/2024" ou "03/15/2024" → "[DATA]"
    text = re.sub(r'\d{1,2}/\d{1,2}/\d{4}', '[DATA]', text)
    
    # PATTERN 2: Remove datas no formato ISO (YYYY-MM-DD)
    # Exemplo: "2024-03-15" → "[DATA]"
    text = re.sub(r'\d{4}-\d{2}-\d{2}', '[DATA]', text)
    
    # PATTERN 3: Remove IDs de pacientes (formato "ID: 12345" ou "Patient ID: 12345")
    # Exemplo: "ID: 12345" → "ID: [PACIENTE_ID]"
    # flags=re.IGNORECASE torna a busca case-insensitive (maiúsculas/minúsculas)
    text = re.sub(r'ID:\s*\d+', 'ID: [PACIENTE_ID]', text, flags=re.IGNORECASE)
    text = re.sub(r'Patient ID:\s*\d+', 'Patient ID: [PACIENTE_ID]', text, flags=re.IGNORECASE)
    
    # PATTERN 4: Remove números de telefone (formato XXX-XXX-XXXX ou XXX.XXX.XXXX)
    # Exemplo: "11987654321" ou "11-98765-4321" → "[TELEFONE]"
    text = re.sub(r'\d{3}[-.]?\d{3}[-.]?\d{4}', '[TELEFONE]', text)
    
    # PATTERN 5: Remove endereços de email
    # Exemplo: "email@hospital.com" → "[EMAIL]"
    # \b garante que estamos no início/fim de uma palavra
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '[EMAIL]', text)
    
    return text

# ============================================================================
# TESTE DA FUNÇÃO DE ANONIMIZAÇÃO
# ============================================================================
# Testa a função com um exemplo real para verificar se está funcionando
# corretamente antes de aplicar em todo o dataset

test_text = "Paciente ID: 12345 foi atendido em 15/03/2024. Contato: 11987654321 ou email@hospital.com"

print("=" * 80)
print("🔒 TESTE DE ANONIMIZAÇÃO")
print("=" * 80)
print("Texto original:")
print(f"  {test_text}")
print("\nTexto anonimizado:")
print(f"  {anonymize_text(test_text)}")
print("=" * 80)
print("✅ Função de anonimização testada e pronta para uso!")


In [ ]:
# ============================================================================
# ETAPA 3: FORMATAÇÃO PARA INSTRUCTION TUNING
# ============================================================================
# Esta etapa transforma os dados médicos brutos em formato de instrução
# adequado para fine-tuning de modelos LLM (LLaMA, Falcon, etc.)
#
# O que é Instruction Tuning?
#   - Técnica de fine-tuning onde o modelo aprende a seguir instruções
#   - Formato: INSTRUÇÃO → ENTRADA → SAÍDA ESPERADA
#   - Permite que o modelo aprenda a responder perguntas baseadas em contexto
#
# Formato usado (inspirado nos notebooks de referência):
#   [|Contexto|] ... [|eContexto|]  → Delimitadores para contexto médico
#   [|Pergunta|] ... [|ePergunta|]  → Delimitadores para pergunta
#   [|Resposta|] ... [|eResposta|]   → Delimitadores para resposta esperada
#
# IMPORTANTE: Esta função deve ser executada DEPOIS de definir anonymize_text()
#             pois ela usa essa função para proteger dados sensíveis

def prepare_medical_instruction(data_id, content):
    """
    Prepara uma entrada do dataset no formato de instrução para fine-tuning
    
    Esta função:
    1. Extrai QUESTION, CONTEXTS, LONG_ANSWER e MESHES do conteúdo
    2. Une múltiplos contextos em um único bloco de texto
    3. Aplica anonimização em contextos e respostas
    4. Formata tudo em um prompt estruturado com delimitadores
    
    Args:
        data_id: ID único do artigo PubMed (chave do dicionário original)
        content: Dicionário com QUESTION, CONTEXTS, LONG_ANSWER, MESHES, etc.
        
    Returns:
        Dicionário com:
          - "id": ID do artigo
          - "input": String formatada pronta para fine-tuning
    """
    # PASSO 1: Extrai os campos principais do conteúdo médico
    question = content.get("QUESTION", "")           # Pergunta médica a ser respondida
    contexts = content.get("CONTEXTS", [])            # Lista de contextos científicos (evidências)
    long_answer = content.get("LONG_ANSWER", "")      # Resposta longa baseada nas evidências
    meshes = content.get("MESHES", [])               # Termos técnicos (Medical Subject Headings)
    
    # PASSO 2: Consolida múltiplos contextos em um único bloco de texto
    # O dataset pode ter vários contextos (fragmentos de artigos científicos)
    # Juntamos todos em uma única string para facilitar o aprendizado do modelo
    context_str = " ".join(contexts)
    
    # PASSO 3: Formata termos MESH como string separada por vírgulas
    # MESH são termos técnicos que ajudam o modelo a entender o domínio
    # Exemplo: ["Mitochondria", "Apoptosis", "Cell Differentiation"]
    meshes_str = ", ".join(meshes) if meshes else ""
    
    # PASSO 4: Constrói o prompt formatado seguindo o padrão dos notebooks de referência
    # Formato inspirado em: [|News|]...[|eNews|] adaptado para medicina
    # 
    # Estrutura do prompt:
    #   INSTRUÇÃO MÉDICA: [instrução geral]
    #   [|Contexto|] [contextos anonimizados] [|eContexto|]
    #   [|Termos|] [termos MESH] [|eTermos|]  (opcional)
    #   [|Pergunta|] [pergunta] [|ePergunta|]
    #   [|Resposta|] [resposta anonimizada] [|eResposta|]
    
    formatted_input = (
        f"INSTRUÇÃO MÉDICA: Responda à pergunta baseando-se nos contextos fornecidos.\n"
        f"[|Contexto|] {anonymize_text(context_str)}[|eContexto|]\n"
    )
    
    # Adiciona termos MESH apenas se existirem (opcional, mas enriquece o contexto)
    if meshes_str:
        formatted_input += f"[|Termos|] {meshes_str}[|eTermos|]\n"
    
    # Adiciona pergunta e resposta (a resposta é anonimizada para proteção de dados)
    formatted_input += (
        f"[|Pergunta|] {question}[|ePergunta|]\n\n"
        f"[|Resposta|]{anonymize_text(long_answer)}[|eResposta|]"
    )
    
    # Retorna estrutura padronizada para o dataset de fine-tuning
    return {
        "id": data_id,
        "input": formatted_input
    }

# ============================================================================
# TESTE DA FUNÇÃO DE FORMATAÇÃO
# ============================================================================
# Testa a função com uma entrada real do dataset para verificar o formato
# antes de processar todas as entradas (economiza tempo e recursos)

sample_id = list(raw_data.keys())[0]      # Pega o primeiro ID disponível
sample_content = raw_data[sample_id]      # Obtém o conteúdo desse ID
sample_processed = prepare_medical_instruction(sample_id, sample_content)  # Processa

print("=" * 80)
print("📝 EXEMPLO DE ENTRADA PROCESSADA")
print("=" * 80)
print(f"ID do artigo: {sample_processed['id']}")
print(f"\nFormato do input (primeiros 500 caracteres):")
print("-" * 80)
print(sample_processed["input"][:500] + "...")
print("-" * 80)
print(f"Tamanho total do input: {len(sample_processed['input'])} caracteres")
print("=" * 80)
print("✅ Função de formatação testada e pronta para processar todo o dataset!")


In [ ]:
# ============================================================================
# ETAPA 4: PROCESSAMENTO COMPLETO DO DATASET
# ============================================================================
# Esta etapa processa TODAS as entradas do dataset original, aplicando:
#   - Anonimização de dados sensíveis
#   - Formatação em estrutura de instrução
#   - Tratamento de erros para garantir robustez
#
# IMPORTANTE: Esta etapa pode levar vários minutos dependendo do tamanho
#             do dataset. Para datasets grandes, considere processar em lotes.
#
# IMPORTANTE: Execute esta célula APENAS DEPOIS de:
#             1. Carregar o dataset (Etapa 1)
#             2. Definir anonymize_text() (Etapa 2)
#             3. Definir prepare_medical_instruction() (Etapa 3)

# Inicializa lista vazia para armazenar dados processados
processed_data = []

# Exibe informações sobre o processamento que será realizado
total_entries = len(raw_data)
print("=" * 80)
print("🔄 INICIANDO PROCESSAMENTO COMPLETO DO DATASET")
print("=" * 80)
print(f"Total de entradas a processar: {total_entries}")
print(f"Este processo pode levar alguns minutos...")
print("-" * 80)

# Itera sobre cada entrada do dataset original
# raw_data.items() retorna pares (id, conteúdo) para cada artigo médico
for data_id, content in raw_data.items():
    try:
        # Aplica a função de formatação definida na Etapa 3
        # Esta função já inclui anonimização automaticamente
        entry = prepare_medical_instruction(data_id, content)
        
        # Adiciona a entrada processada à lista
        processed_data.append(entry)
        
        # Exibe progresso a cada 1000 entradas processadas (opcional, para feedback)
        if len(processed_data) % 1000 == 0:
            progress = (len(processed_data) / total_entries) * 100
            print(f"Progresso: {len(processed_data)}/{total_entries} ({progress:.1f}%)")
            
    except Exception as e:
        # Tratamento de erros: se uma entrada falhar, registra o erro mas continua
        # Isso garante que o processamento não pare por causa de uma entrada problemática
        print(f"⚠️  Erro ao processar entrada {data_id}: {e}")
        continue

# Exibe resumo final do processamento
print("-" * 80)
print("=" * 80)
print("✅ PROCESSAMENTO CONCLUÍDO")
print("=" * 80)
print(f"Entradas processadas com sucesso: {len(processed_data)}")
print(f"Taxa de sucesso: {(len(processed_data)/total_entries)*100:.2f}%")
print(f"Entradas com erro: {total_entries - len(processed_data)}")
print("=" * 80)


In [ ]:
# ============================================================================
# ETAPA 5: SALVAMENTO DO DATASET DE TREINO
# ============================================================================
# Esta etapa salva o dataset processado em um arquivo JSON que será usado
# diretamente no processo de fine-tuning do modelo LLM
#
# Formato do arquivo de saída:
#   [
#     {
#       "id": "21645374",
#       "input": "INSTRUÇÃO MÉDICA: ... [|Contexto|] ... [|eContexto|] ..."
#     },
#     ...
#   ]
#
# IMPORTANTE: Execute esta célula APENAS DEPOIS de processar todos os dados (Etapa 4)
#             Caso contrário, o arquivo será salvo vazio ou incompleto

# Define o nome do arquivo de saída
# Este arquivo será criado no mesmo diretório do notebook
output_file = 'medical_tuning_data.json'

print("=" * 80)
print("💾 SALVANDO DATASET PROCESSADO")
print("=" * 80)
print(f"Arquivo de destino: {output_file}")
print(f"Total de entradas a salvar: {len(processed_data)}")
print("-" * 80)

# Abre o arquivo em modo de escrita ('w') com encoding UTF-8
# UTF-8 é necessário para suportar caracteres especiais e acentos
with open(output_file, 'w', encoding='utf-8') as f:
    # json.dump() escreve a lista de dados processados no arquivo
    # indent=2: formata o JSON com indentação de 2 espaços (melhora legibilidade)
    # ensure_ascii=False: preserva caracteres não-ASCII (acentos, etc.)
    json.dump(processed_data, f, indent=2, ensure_ascii=False)

# Verifica se o arquivo foi criado e exibe informações
import os
file_size = os.path.getsize(output_file) / (1024 * 1024)  # Tamanho em MB

print("✅ Dataset salvo com sucesso!")
print(f"   Arquivo: {output_file}")
print(f"   Total de entradas: {len(processed_data)}")
print(f"   Tamanho do arquivo: {file_size:.2f} MB")
print("=" * 80)
print("\n📌 PRÓXIMOS PASSOS:")
print("   1. Valide os dados com: python validate_data.py")
print("   2. Use este arquivo para fine-tuning com Hugging Face, PEFT, etc.")
print("   3. Configure hiperparâmetros de treinamento apropriados")
print("=" * 80)


In [ ]:
# ============================================================================
# ETAPA 6: VERIFICAÇÃO FINAL E VISUALIZAÇÃO
# ============================================================================
# Esta etapa exibe uma amostra das entradas processadas para verificação
# visual da qualidade e formato dos dados
#
# Por que verificar?
#   - Confirma que o formato está correto
#   - Permite identificar problemas visuais antes do treinamento
#   - Ajuda a entender como os dados serão usados pelo modelo
#
# IMPORTANTE: Esta é uma etapa opcional, mas recomendada para garantir
#             qualidade antes de iniciar o fine-tuning

print("=" * 80)
print("🔍 VERIFICAÇÃO FINAL - AMOSTRA DE DADOS PROCESSADOS")
print("=" * 80)
print(f"Exibindo as primeiras 3 entradas do dataset processado")
print(f"(Total de {len(processed_data)} entradas disponíveis)")
print("=" * 80)

# Itera sobre as primeiras 3 entradas processadas
# [:3] pega apenas os primeiros 3 elementos da lista
for i, entry in enumerate(processed_data[:3], start=1):
    print(f"\n📄 ENTRADA {i} de 3")
    print("-" * 80)
    print(f"ID do artigo: {entry['id']}")
    print(f"Tamanho do input: {len(entry['input'])} caracteres")
    print(f"\nPreview do input (primeiros 300 caracteres):")
    print("-" * 80)
    print(entry['input'][:300] + "...")
    print("-" * 80)

print("\n" + "=" * 80)
print("✅ Verificação concluída!")
print("=" * 80)
print("\n💡 DICAS:")
print("   - Verifique se os delimitadores [|Contexto|], [|Pergunta|], etc. estão presentes")
print("   - Confirme que dados sensíveis foram anonimizados corretamente")
print("   - Valide que o formato está consistente entre todas as entradas")
print("=" * 80)
